In [7]:
# Gelişmiş Analizler
# Eksik Değerlerin Bağımlı Değişken ile İlişkisinin İncelenmesi

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install missingno
#import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder,StandardScaler,RobustScaler

In [8]:
def load():
    data = pd.read_csv("titanic.csv")
    return data

df = load()

def missing_values_table(df,na_name=False):
    na_columns = [col for col in df.columns if df[col].isnull().sum()>0] # Eksik değerleri seçtik
    
    n_miss = df[na_columns].isnull().sum().sort_values(ascending=False) # Sırala
    
    ratio = (df[na_columns].isnull().sum() / df.shape[0] * 100).sort_values(ascending=False) # Oran
    
    missing_df = pd.concat([n_miss, np.round(ratio,2)], axis=1, keys=["n_miss","ratio"]) # missing_df adında bir df'de birleştir
    
    print(missing_df,end="\n")
    
    if na_name: # İsimleri ver 
        return na_columns

In [9]:
na_cols = missing_values_table(df,True)

          n_miss  ratio
Cabin        687  77.10
Age          177  19.87
Embarked       2   0.22


In [13]:
def missing_vs_target(df,target,na_columns):
    temp_df = df.copy()
    for col in na_columns:
        temp_df[col+"_NA_FLAG"] = np.where(temp_df[col].isnull(),1,0) # -> n/a'ya sahip sutünlarda gez ve kopyaya ekle 
                                                                      # Eksiklik gördüğün yere 1 diğerine 0 yaz.
    
    na_flags = temp_df.loc[:,temp_df.columns.str.contains("_NA_")].columns
    # İçinde NA ifadesi olan sutünları getir
    
    for col in na_flags:
        print(pd.DataFrame({"TARGET_MEAN" : temp_df.groupby(col)[target].mean(),
                           "Count": temp_df.groupby(col)[target].count()}), end="\n\n\n")
        # Target'a göre ortalama al
        # Target'a göre toplam getir

In [14]:
# Karşılaştırma Yapıyoruz. Birbiri ile ilişkisi var mı?
missing_vs_target(df,"Survived",na_cols)

             TARGET_MEAN  Count
Age_NA_FLAG                    
0               0.406162    714
1               0.293785    177


               TARGET_MEAN  Count
Cabin_NA_FLAG                    
0                 0.666667    204
1                 0.299854    687


                  TARGET_MEAN  Count
Embarked_NA_FLAG                    
0                    0.382452    889
1                    1.000000      2


